In [7]:
%load_ext autoreload
%autoreload 2
%matplotlib inline  

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

import hervpd as hp
import csvUtils as csvu
import parseActivityFiles as paf
import parseIntervalFiles as pif

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
    user = 0
    RAW_DATA_PATH = "C:\\Users\\Ju\\Google Drive\\Projects\\HeRV\\Data\\Raw\\%d"%user
    PRE_DATA_PATH = "C:\\Users\\Ju\\Google Drive\\Projects\\HeRV\\Data\\PreProcessed\\%d"%user
    verbose = True

In [3]:
sessions = paf.parseActivityFiles(dirname=RAW_DATA_PATH)

reading act170929.csv ... 
orphan stop in: ['2017-09-29 00:19:00', 'stop', '', '', '', '', '']
reading act170930.csv ... 
reading act171001.csv ... 
reading act171003.csv ... 
orphan start in: ['2017-10-03 22:56:19', 'start', 'eat', 'sitting']
reading act171004.csv ... 
orphan start in: ['2017-10-04 13:31:16', 'start', 'movement', 'standing']
orphan start in: ['2017-10-04 14:59:24', 'start', 'rest-passive', 'sitting']
orphan start in: ['2017-10-04 15:14:23', 'start', 'movement', 'standing']
orphan start in: ['2017-10-04 17:05:36', 'start', 'rest-passive', 'sitting']
orphan start in: ['2017-10-04 23:30:33', 'start', 'sleep', 'lying down']
reading act171005.csv ... 
orphan start in: ['2017-10-05 11:08:11', 'start', 'focused-passive', 'sitting', '']
orphan start in: ['2017-10-05 12:54:00', 'start', 'eat', 'sitting', '']
orphan start in: ['2017-10-05 13:48:39', 'start', 'rest-active', 'sitting', '']
orphan start in: ['2017-10-05 13:57:37', 'start', 'movement', 'standing', '']
orphan start 

In [9]:
for sess in sessions:        
    sess['duration'] = int((sess['stop']-sess['start']).seconds)        
    sess['rr'] = pif.getIntervals(sess['start'], sess['stop'], RAW_DATA_PATH)    

In [10]:
df = pd.DataFrame(sessions)
df.head()

,activity,duration,notes,posture,rr,start,stop
0,rest-active,360,break,standing,[],2017-09-29 00:19:00,2017-09-29 00:25:00
1,focused-active,240,despair over lost file,sitting,[],2017-09-29 00:26:00,2017-09-29 00:30:00
2,leisure-active,2040,playing,lying down,[],2017-09-29 00:32:00,2017-09-29 01:06:00
3,sleep,18600,,lying down,[],2017-09-29 01:06:00,2017-09-29 06:16:00
4,movement,2880,walking,standing,[],2017-09-29 06:21:00,2017-09-29 07:09:00
